# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [1]:
import io
import re
from collections import defaultdict
import json

class BagOfWords():
    def __init__(self, obj):
        text = obj if isinstance(obj, str) else obj.read()
        text = re.sub('[^a-zA-Z]+', ' ', text)
        text = text.lower()
        
        self.word_counter = defaultdict(int)
        for word in text.split():
            self.word_counter[word] += 1
        
    def __getitem__(self, key):
        return self.word_counter[key]
    
    def __setitem__(self, key, value):
        self.word_counter[key] = value
        
    def __str__(self):
        return ', '.join(f"{k}:{v}" for k, v in self.word_counter.items())
    
    def __contains__(self, key):
        return key in self._d
    
    def __iter__(self):
        return iter(sorted(self.word_counter, key=self.word_counter.get, reverse=True))
    
    def __next__(self):
        if self.index == len(self.word_counter):
            raise StopIteration
        val = list(self.word_counter)[self.index]
        self.index = self.index + 1
        return val
    
    def __add__(self, second_bag):
        new_bag = BagOfWords()
        new_bag._d = self.add(self.word_counter, second_bag.word_counter)
        return new_bag
            
    def __eq__(self, other):
        return self.__dict__ == other.__dict__
            
    @staticmethod
    def add(d1, d2):
        return defaultdict(int, {**d1, **d2, **{k: d1[k] + d2[k] for k in set(d1) & set(d2)}})
    
    


In [2]:
hamlet_bag = BagOfWords(open('hamlet.txt'))
print(hamlet_bag["hamlet"], "\n")

for i, key in enumerate(hamlet_bag):
    if i < 10:
        print(key)
    else: break

121 

the
and
to
of
i
you
a
my
in
it


## Zadanie 3

Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [3]:
import pickle

with open('pickle_bow', 'wb+') as f:
    pickle.dump(hamlet_bag, f)

with open('pickle_bow', 'rb') as f:
    hamlet_bag_pickled = pickle.load(f)

print(hamlet_bag == hamlet_bag_pickled)
print(dir(hamlet_bag) == dir(hamlet_bag_pickled))

    

True
True


## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

In [5]:
import time
from datetime import datetime, timedelta
from pytz import timezone
from collections import defaultdict
from countryinfo import countries

countries_by_continent = defaultdict(list)
for country in countries:
    countries_by_continent[country["continent"]].append(country)

for continent, countries in countries_by_continent.items():
    print(f"***** {continent} *****")
    for country in countries:
        print(f"\t{country['name']}:")
        for tz in country["timezones"]:
            tz_obj = timezone(tz)
            dt = datetime.now(tz_obj)            
            print(f"\t\tTime: {dt.strftime('%H:%M:%S %d-%m-%Y')}, Timezone: {tz}: ")
    print()



***** Europe *****
	Andorra:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Andorra: 
	Albania:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Tirane: 
	Austria:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Vienna: 
	Belgium:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Brussels: 
	Bulgaria:
		Time: 01:28:47 02-12-2020, Timezone: Europe/Sofia: 
	Belarus:
		Time: 02:28:47 02-12-2020, Timezone: Europe/Minsk: 
	Czech Republic:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Prague: 
	Germany:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Berlin: 
	Denmark:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Copenhagen: 
	Estonia:
		Time: 01:28:47 02-12-2020, Timezone: Europe/Tallinn: 
	Finland:
		Time: 01:28:47 02-12-2020, Timezone: Europe/Helsinki: 
	France:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Paris: 
	Greece:
		Time: 01:28:47 02-12-2020, Timezone: Europe/Athens: 
	Hungary:
		Time: 00:28:47 02-12-2020, Timezone: Europe/Budapest: 
	Republic of Ireland:
		Time: 23:28:47 01-12-2020, Timez

In [ ]:
def seconds_to_midnight(dt):
    tomorrow = dt + timedelta(days=1)
    return ((24 - dt.hour - 1) * 60 * 60) + ((60 - dt.minute - 1) * 60) + (60 - dt.second)

timezones = set([(tz, c["name"]) for c in countries for tz in c["timezones"]])

sec_to_midnight = defaultdict(list)
for tz, c in timezones:
    tz_obj = timezone(tz)
    dt = datetime.now(tz_obj)
    midnight = seconds_to_midnight(dt)
    sec_to_midnight[midnight].append(c)


sec_to_midnight_sorted = sorted(sec_to_midnight.items(), key= lambda item: item[0])     
for i, (sec, countries) in enumerate(sec_to_midnight_sorted):
    sec_to_sleep = sec if i == 0 else sec - sec_to_midnight_sorted[i-1][0]
    print(f"Wait {str(sec_to_sleep)} seconds until next cities reach midnight")
    time.sleep(sec_to_sleep)
    for country in countries:
        print(country)
        
midnight_symulation(countries)


Wait 37861 seconds until next cities reach midnight


## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.

In [1]:
import io
import re
from collections import defaultdict
import json

class BagOfWords():
    def __init__(self, obj):
        text = obj if isinstance(obj, str) else obj.read()
        text = re.sub('[^a-zA-Z]+', ' ', text)
        text = text.lower()
        
        self.word_counter = defaultdict(int)
        for word in text.split():
            self.word_counter[word] += 1
        
    def __getitem__(self, key):
        return self.word_counter[key]
    
    def __setitem__(self, key, value):
        self.word_counter[key] = value
        
    def __str__(self):
        return ', '.join(f"{k}:{v}" for k, v in self.word_counter.items())
    
    def __contains__(self, key):
        return key in self._d
    
    def __iter__(self):
        return iter(sorted(self.word_counter, key=self.word_counter.get, reverse=True))
    
    def __next__(self):
        if self.index == len(self.word_counter):
            raise StopIteration
        val = list(self.word_counter)[self.index]
        self.index = self.index + 1
        return val
    
    def __add__(self, second_bag):
        new_bag = BagOfWords()
        new_bag._d = self.add(self.word_counter, second_bag.word_counter)
        return new_bag
    
    def save(self, path):
        with open(path, mode='w') as f:
            json.dump({"word_counter": self.word_counter}, f)
        
    def load(self, path):
        with open(path, mode='r') as f:
            json_content = json.load(f)
            self.word_counter = json_content["word_counter"]
            
    def __eq__(self, other):
        return self.__dict__ == other.__dict__
            
    @staticmethod
    def add(d1, d2):
        return defaultdict(int, {**d1, **d2, **{k: d1[k] + d2[k] for k in set(d1) & set(d2)}})

In [2]:
hamlet_bag = BagOfWords(open('hamlet.txt'))
hamlet_bag.save("hamlet.json")

hamlet_bag2 = BagOfWords('')
hamlet_bag2.load("hamlet.json")

print()
print(hamlet_bag2["hamlet"])


121
